# Test name: qa_test_08

##### Reason for test:  
To test more complicated distributions where there are three supply points and demand points in a ring surrounding it - in this case the middle supply can only support two demand points leaving the others to be redistributed to the other supplies which are large enough to take on the extra demand.

##### Parameters

Three supplies - vertically aligned - 150km apart

8 demands for each supply - in an even circle - a fixed distance away (50km)

Capacities - A - supply capable of sustaining 12 demand points, B - supply capable of sustaining 2 demand points, C - supply capable of sustaining 12 demand points.

##### Predicted outcome:
For the centre supply (B) - the two demands horizontally aligned should be attached to it with the others being reassigned depending on whether they are to the North or South of this horizontal line



In [1]:
import pandas as pd
import psycopg2
import random
import os
from utilities import generate_point_dict 
from utilities import locate_supplies
from utilities import calculate_distances
dir = os.getcwd()
parent_directory = os.path.split(os.path.split(dir)[0])[0]

An example of what the csv data should look like (see column naming)

In [5]:
pd.read_csv(parent_directory + "/web/data/datasets/data.csv", nrows=2)

,demand,demand_id,demand_lat,demand_lng,demand_name,supply,supply_id,supply_lat,supply_lng,supply_name,duration_min,distance_crowflies_km,distance_route_km
0,11.662019,0,50.920128,-2.670739,Green Ln,35.689500,0,51.749314,-0.240863,Roehyde Way,158.883333,192.761855,235.458
1,11.662019,0,50.920128,-2.670739,Green Ln,28.249443,1,50.815128,-2.273901,Deer Park,48.433333,30.269113,50.560


The method for locating demands will change depending on the scenario and the locate_demands function may be different for each different test.

In [6]:
def locate_demands(supply_df, num_demands_per_supply, dem_distances_km, dem_bearings_degrees, demand_vol, conn):
    demands = []
    supply_counter = 0

    for supply_dict in supply_df.to_dict(orient="records"):
        this_supply = {"lat":supply_dict["supply_lat"], "lng": supply_dict["supply_lng"]}
        for i in range(num_demands_per_supply):
            distance_km = random.choice(dem_distances_km)
            bearing_degrees = dem_bearings_degrees[i]
            this_demand = generate_point_dict(this_supply, bearing_degrees, distance_km, conn)
            this_demand["demand"] = random.choice(demand_vol)
            this_demand["demand_name"] = "demand" + str(supply_counter + i)
            demands.append(this_demand)
        supply_counter = supply_counter + num_demands_per_supply

    demands_df = pd.DataFrame(demands).reset_index()
    demands_df.columns=["demand_id", "demand", "demand_name", "demand_lat", "demand_lng"]
    return(demands_df)

### Run from here to create a new test data set based on the parameters which can be changed below.

In [7]:
# Naming parameters
output_name = "qa_data_08"

# Supply parameters
num_supplies = 3
sup_distances_km = [0, 150, 300]
sup_bearings_degrees = [0, 0, 0]
supply_vol = [1200, 200, 1200]

# Demand parameters
num_demands_per_supply = 8
dem_distances_km = [50, 50, 50, 50, 50, 50, 50, 50]
dem_bearings_degrees = [0, 45, 90, 135, 180, 225, 270, 315] # Demands will be taken at these exact degrees for each supply
demand_vol = [100]

# Misc
scale = 1 # Used to scale demand to be a proportion of supply (1 by default)

In [8]:
conn = "host='localhost' dbname='postgres' user='postgres' password=''"
conn = psycopg2.connect(conn)

supply_df = locate_supplies(num_supplies, sup_distances_km, sup_bearings_degrees, supply_vol, conn)
demands_df = locate_demands(supply_df, num_demands_per_supply, dem_distances_km, dem_bearings_degrees, demand_vol, conn)


In [9]:
supply_df

,supply_id,supply_lat,supply_lng,supply,supply_name
0,0,51.501109,-1.242375,1200,supply0
1,1,52.849173,-1.242375,200,supply1
2,2,54.196930,-1.242375,1200,supply2


In [10]:
demands_df

,demand_id,demand,demand_name,demand_lat,demand_lng
0,0,100,demand0,51.950498,-1.242375
1,1,100,demand1,51.817765,-0.729654
2,2,100,demand2,51.498899,-0.522340
3,3,100,demand3,51.182226,-0.736736
4,4,100,demand4,51.051685,-1.242375
5,5,100,demand5,51.182226,-1.748014
6,6,100,demand6,51.498899,-1.962410
7,7,100,demand7,51.817765,-1.755096
8,8,100,demand8,53.298460,-1.242375
9,9,100,demand9,53.165701,-0.713734


In [10]:
# Scale supply capacity so that supply>demand by "scale" (ONLY RUN IF SCALING REQUIRED)
#supply_df["supply"] = supply_df["supply"]* demands_df["demand"].sum()/supply_df["supply"].sum()*scale

In [11]:
# Cartestian product
supply_df["cart"] = 1
demands_df["cart"] = 1
all_combos = demands_df.merge(supply_df)
all_combos = all_combos.drop("cart", axis=1)
all_combos

,demand_id,demand,demand_name,demand_lat,demand_lng,supply_id,supply_lat,supply_lng,supply,supply_name
0,0,100,demand0,51.950498,-1.242375,0,51.501109,-1.242375,1200,supply0
1,0,100,demand0,51.950498,-1.242375,1,52.849173,-1.242375,200,supply1
2,0,100,demand0,51.950498,-1.242375,2,54.196930,-1.242375,1200,supply2
3,1,100,demand1,51.817765,-0.729654,0,51.501109,-1.242375,1200,supply0
4,1,100,demand1,51.817765,-0.729654,1,52.849173,-1.242375,200,supply1
5,1,100,demand1,51.817765,-0.729654,2,54.196930,-1.242375,1200,supply2
6,2,100,demand2,51.498899,-0.522340,0,51.501109,-1.242375,1200,supply0
7,2,100,demand2,51.498899,-0.522340,1,52.849173,-1.242375,200,supply1
8,2,100,demand2,51.498899,-0.522340,2,54.196930,-1.242375,1200,supply2
9,3,100,demand3,51.182226,-0.736736,0,51.501109,-1.242375,1200,supply0


In [12]:
# Calculate the crowflies distances for each combination
all_combos = calculate_distances(all_combos, conn)
all_combos.head(10)

,demand_id,demand,demand_name,demand_lat,demand_lng,supply_id,supply_lat,supply_lng,supply,supply_name,duration_min,distance_crowflies_km,distance_route_km
0,0,100,demand0,51.950498,-1.242375,0,51.501109,-1.242375,1200,supply0,49.983150,49.983150,49.983150
1,0,100,demand0,51.950498,-1.242375,1,52.849173,-1.242375,200,supply1,99.966210,99.966210,99.966210
2,0,100,demand0,51.950498,-1.242375,2,54.196930,-1.242375,1200,supply2,249.915294,249.915294,249.915294
3,1,100,demand1,51.817765,-0.729654,0,51.501109,-1.242375,1200,supply0,49.984545,49.984545,49.984545
4,1,100,demand1,51.817765,-0.729654,1,52.849173,-1.242375,200,supply1,119.934313,119.934313,119.934313
5,1,100,demand1,51.817765,-0.729654,2,54.196930,-1.242375,1200,supply2,266.911016,266.911016,266.911016
6,2,100,demand2,51.498899,-0.522340,0,51.501109,-1.242375,1200,supply0,49.985289,49.985289,49.985289
7,2,100,demand2,51.498899,-0.522340,1,52.849173,-1.242375,200,supply1,158.065716,158.065716,158.065716
8,2,100,demand2,51.498899,-0.522340,2,54.196930,-1.242375,1200,supply2,304.044870,304.044870,304.044870
9,3,100,demand3,51.182226,-0.736736,0,51.501109,-1.242375,1200,supply0,49.984577,49.984577,49.984577


In [13]:
# Save file to CSV
path = dir + "/qa_data_files/" + str(output_name) + ".csv"
all_combos.to_csv(path, index=False)